In [25]:
import warnings,subprocess, os, pandas as pd, geopandas as gpd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, folium
from IPython.display import display
from run_analysis import buffer_list_mile, ctrl_vars
from classes import factor_loading_matrices
%matplotlib inline
np.set_printoptions(suppress=True)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", "{:.2f}".format)
nb = os.path.basename(__file__) if "__file__" in globals() else "spatial_analysis.ipynb"
subprocess.run(["jupyter", "trust", nb])

CompletedProcess(args=['jupyter', 'trust', 'spatial_analysis.ipynb'], returncode=0)

**User input.**


In [17]:
pca_factors_network = 4
pca_factors_ctrl = 9 

**Loads spatial or tabular datasets into memory.**


In [18]:
gdf = gpd.read_file('../data/neighborhoods/Final_dataset.shp');gdf.head()

,id,propcrim1k,propcrimct,area,area_acre,prpcrmarea,bachdg,emp,home_own,resoccup,...,shtpth075,st_den10,int_den10,nd_deg10,cl_cof10,cl_cnt10,pr_cnt10,bt_cnt10,shtpth10,geometry
0,2,60.20,129,17917968.29,411.34,0.31,47.98,95.49,38.69,94.02,...,1.88,123.11,0.13,2.56,0.04,0.05,0.00,0.03,2.08,"POLYGON ((1467289.788 535340.984, 1466444.783 ..."
1,3,60.39,732,50253198.39,1153.65,0.63,83.46,97.64,38.90,86.45,...,1.81,162.69,0.19,2.94,0.04,0.05,0.00,0.02,2.10,"POLYGON ((1452274.574 537616.615, 1452086.651 ..."
2,4,6.85,8,14442159.23,331.55,0.02,89.47,98.04,100.00,89.69,...,1.72,111.18,0.10,2.43,0.04,0.06,0.00,0.03,1.98,"POLYGON ((1458078.129 520074.187, 1458353.702 ..."
3,5,25.91,20,7290342.73,167.36,0.12,1.95,84.72,23.18,93.95,...,1.50,148.16,0.16,2.59,0.04,0.06,0.00,0.02,1.78,"POLYGON ((1437260.245 553987.106, 1437745.545 ..."
4,6,80.69,145,17649740.61,405.18,0.36,22.00,100.00,31.46,87.99,...,1.77,150.57,0.16,2.65,0.04,0.05,0.00,0.02,2.01,"POLYGON ((1439748.778 545058.078, 1438983.548 ..."


In [19]:
def generate_street_network_columns(buffer_list_mile):
    # Prefixes for all network metrics
    metrics = ["int_den", "st_den", "nd_deg", "cl_cof", "shtpth", "bt_cnt", "cl_cnt", "pr_cnt"]

    # Same naming rule used in StreetNetworkProcessor
    def bufname(b):
        return str(b).replace(".", "") if (b < 1 and b != 0.1) else str(int(b * 10))

    # Generate all columns
    return [f"{m}{bufname(b)}" for b in buffer_list_mile for m in metrics]

In [20]:
final_list_net = generate_street_network_columns(buffer_list_mile)

**Generates visualization plots for diagnostics or distributions**


In [21]:
popup_vars = ["id"] + final_list_net + ctrl_vars
gdf_vis = gdf.to_crs(4326)

def fmt(v):
    return f"{v:.2f}" if isinstance(v, (float, int)) and pd.notna(v) else v

# Build FeatureCollection
features = []
for _, r in gdf_vis.iterrows():
    popup_html = "<br>".join([f"{v}: {fmt(r.get(v, 'NA'))}" for v in popup_vars])
    features.append({
        "type": "Feature",
        "geometry": r.geometry.__geo_interface__,
        "properties": {"id": r["id"], "popup": popup_html}
    })

geojson = {"type": "FeatureCollection", "features": features}

# Create map
m = folium.Map(
    [gdf_vis.geometry.centroid.y.mean(), gdf_vis.geometry.centroid.x.mean()],
    zoom_start=11, tiles="CartoDB positron"
)

folium.GeoJson(
    geojson,
    tooltip=folium.GeoJsonTooltip(fields=["id"], aliases=["ID:"]),
    popup=folium.GeoJsonPopup(fields=["popup"], labels=False, max_width=350)
).add_to(m)

m

**Performs exploratory data inspection.**


In [22]:
print("📊 Descriptive Statistics\n")
display(gdf[final_list_net + ctrl_vars].describe().T.round(3))

📊 Descriptive Statistics



,count,mean,std,min,25%,50%,75%,max
int_den025,371.00,0.11,0.04,0.03,0.08,0.10,0.12,0.27
st_den025,371.00,113.73,29.92,38.88,95.18,111.65,129.66,211.79
nd_deg025,371.00,2.34,0.17,2.04,2.22,2.32,2.44,2.95
cl_cof025,371.00,0.03,0.02,0.00,0.02,0.03,0.04,0.12
shtpth025,371.00,1.31,0.38,0.64,1.05,1.26,1.52,3.07
bt_cnt025,371.00,0.06,0.01,0.02,0.04,0.05,0.07,0.12
cl_cnt025,371.00,0.09,0.02,0.05,0.08,0.09,0.10,0.15
pr_cnt025,371.00,0.01,0.00,0.00,0.00,0.01,0.01,0.02
int_den05,371.00,0.11,0.04,0.03,0.08,0.10,0.12,0.27
st_den05,371.00,108.39,27.22,36.36,90.71,106.93,121.11,210.30


**Missing values.**

**Note: Any missing values must be handled before PCA analysis.**


In [23]:
missing_values = gdf[final_list_net + ctrl_vars].isnull().sum()
missing_values_filtered = missing_values[missing_values > 0]
if len(missing_values_filtered) == 0:
    print("No missing values in any feature")
else:
    print(missing_values_filtered)

home_own       1
sinfamrate     1
sinfamage     15
sinfamsize    13
salesprice     3
income         6
race_white     1
dtype: int64


**Generates visualization plots for diagnostics or distributions.**


In [ ]:
# Plotting histogram and bar plots of each variable
numerical_columns = gdf[final_list_net + ctrl_vars].select_dtypes(include=['number']).columns

print("histogram and bar plots of each variable")

for col in numerical_columns:
    print(col)
    print('Skew :', round(gdf[final_list_net + ctrl_vars][col].skew(), 2))
    plt.figure(figsize = (10, 4))
    plt.subplot(1, 2, 1)
    sns.distplot(gdf[final_list_net + ctrl_vars][col], axlabel=col)
    plt.subplot(1, 2, 2)
    sns.boxplot(x = gdf[final_list_net + ctrl_vars][col])
    plt.show()

**Cell 9: Generates visualization plots for diagnostics or distributions.**


In [ ]:
# Define the subset of columns
selected_cols = ["id"] + final_list_net + ctrl_vars
sub = gdf[selected_cols]

# Identify numeric columns (excluding the ID if needed)
numerical_columns = sub.select_dtypes(include='number').columns.tolist()

if "id" in numerical_columns:
    numerical_columns.remove("id")

# Plotting boxplots
for col in numerical_columns:
    print(f"Boxplot for {col}")

    plt.figure(figsize=(10, 5))

    # Boxplot
    sns.boxplot(data=sub, x=col)

    # Overlaid raw points
    sns.stripplot(data=sub, x=col, color='black', jitter=True, alpha=0.5)

    # Add ID labels
    for i in range(sub.shape[0]):
        plt.text(
            x=sub[col].iloc[i],
            y=0.02,
            s=str(sub["id"].iloc[i]),
            horizontalalignment='center',
            size='small',
            color='blue',
            rotation=45
        )

    plt.title(f"Boxplot with Observation IDs for {col}")
    plt.tight_layout()
    plt.show()

**Cell 11: Computes correlation matrix for control variables.**


In [ ]:
# Correlation heatmap for control variables (compact)
corr = gdf[ctrl_vars].corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title("Control Variables Correlation")
plt.show()

**Cell 14: Runs PCA on street network variables and extracts components.**


In [ ]:
unrotated_loading_net,rotated_loading_net, communalities_net, unrotated_net_scores, rotated_net_scores, unrot_net_eigenvalues, rotated_variance_share_net = factor_loading_matrices(gdf, final_list_net, n_factors=pca_factors_network)

**Cell 15: Runs PCA on control variables and extracts components.**


In [ ]:
unrotated_loading_ctrl,rotated_loading_ctrl, communalities_ctrl, unrotated_ctrl_scores, rotated_ctrl_scores, unrot_ctrl_eigenvalues, rotated_variance_share_ctrl = factor_loading_matrices(gdf, ctrl_vars, n_factors=pca_factors_ctrl)

In [ ]:
!jupyter nbconvert --to html Spatial_Anaysis.ipynb